从原始数据集中提取出突变，用annovar进行注释

In [ ]:
import pandas as pd

raw_data_file = "..\data\humu22768-sup-0001-suppmat\humu22768-sup-0002-data.xlsx"
df = pd.read_excel(raw_data_file, index_col=0, sheet_name = None, usecols=["CHR", "Nuc-Pos", "REF-Nuc", "ALT-Nuc"], engine="openpyxl")
print(df.keys())

In [ ]:
import os

variant_file_dir = "..\data\\variant"
if not os.path.exists(variant_file_dir):
    os.mkdir(variant_file_dir)
for key in df.keys():
        _ = df[key]
        _.to_csv(os.path.join(variant_file_dir, "{}.csv".format(key)))


convert the snp files to vcf

In [ ]:
import pandas 
import os

variant_files_dir = "../data/variant"
output_dir = "../data/variant"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
print("starting")
for root, dirs, files in os.walk(variant_files_dir):
    for file_ in files:
        path = os.path.join(root, file_)
        print("process {}".format(path))
        df_variants = pandas.read_csv(path, index_col=0)
        print(list(df_variants))
        output_file = file_.split(".")[0] + ".vcf"
        if output_file is None:
            output_file = str(uuid.uuid4())
        output = os.path.join(output_dir, output_file)
        with open(output, "w+") as f:
            f.write("##fileformat=VCFv4.3\n")
            f.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\n")
            for index, row in df_variants.iterrows():
                chr = index
                pos = row["Nuc-Pos"]
                ref = row["REF-Nuc"]
                alt = row["ALT-Nuc"]
                f.write("{}\t{}\t.\t{}\t{}\t.\t.\t.\n".format(chr, pos, ref, alt))

check the output of annovar

In [ ]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt

def annovar_res_visualize(exo_df):
    # exo_df = exo_df.loc[:,~(exo_df == ".").any()]
    cols = list(exo_df)
    stats = [[] for _ in range(3)]
    for col in cols:
        stats[0].append((exo_df[col]==".").sum())
        stats[1].append((exo_df[col]!=".").sum())
        stats[2].append(col)

    plt.figure(figsize=(80, 8))
    plt.xticks(rotation=45)
    size = 5
    plt.bar(range(len(cols)), stats[0], label="NaN", tick_label=stats[2])
    plt
    plt.bar(range(len(cols)), stats[1], label="Non-NaN", bottom = stats[0])
    plt.legend()
    plt.show()

dir_ = "../data/variant"
for root, dirs, files in os.walk(dir_):
    for f in files:
        if f.split(".")[-1] == "txt":
            file_path = os.path.join("../data/variant", f)
            df = pd.read_table(file_path).iloc[:,:-11]
            df.to_csv(os.path.join("../data/variant", ".".join(f.split(".")[:-1]) + ".csv"), index=0)
            annovar_res_visualize(df)


In [37]:
import pandas as pd
import os

datasets = ['HumVarFiltered', 'ExoVarFiltered', 'VariBenchSelected', 'predictSNPSelected', 'SwissVarSelected']
# datasets = ['HumVarFiltered']
dir_ = "../data/variant"
out_dir_ = "../data/variants_selected/"
dfs = []
keys = ['True Label', 'Chr', 'Start', 'End', 'Ref', 'Alt', 'Func.refGene', 'Gene.refGene', 'GeneDetail.refGene', 'ExonicFunc.refGene', 'AAChange.refGene', 'Func.knownGene', 'Gene.knownGene', 'GeneDetail.knownGene', 'ExonicFunc.knownGene', 'AAChange.knownGene', 'Func.ensGene', 'Gene.ensGene', 'GeneDetail.ensGene', 'ExonicFunc.ensGene', 'AAChange.ensGene', 'SIFT_score', 'SIFT_pred', 'Polyphen2_HDIV_score', 'Polyphen2_HDIV_pred', 'Polyphen2_HVAR_score', 'Polyphen2_HVAR_pred', 'LRT_score', 'LRT_pred', 'MutationTaster_score', 'MutationTaster_pred', 'MutationAssessor_score', 'MutationAssessor_pred', 'FATHMM_score', 'FATHMM_pred', 'RadialSVM_score', 'RadialSVM_pred', 'LR_score', 'LR_pred', 'VEST3_score', 'CADD_raw', 'CADD_phred', 'GERP++_RS', 'phyloP46way_placental', 'phyloP100way_vertebrate', 'SiPhy_29way_logOdds', 'Alt.1', '#Chr', 'Start.1', 'DamagePredCount', 'SIFT_pred.1', 'SIFT4G_pred', 'LRT_pred.1', 'MutationTaster_pred.1', 'MutationAssessor_pred.1', 'FATHMM_pred.1', 'PROVEAN_pred', 'MetaSVM_pred', 'MetaLR_pred', 'M-CAP_pred', 'MutPred_score', 'MVP_score', 'MPC_score', 'PrimateAI_pred', 'DEOGEN2_pred', 'BayesDel_addAF_pred', 'BayesDel_noAF_pred', 'LIST-S2_pred', 'DANN_score', 'fathmm-MKL_coding_pred', 'fathmm-XF_coding_pred', 'Eigen-raw_coding', 'Eigen-phred_coding', 'Eigen-PC-raw_coding', 'Eigen-PC-phred_coding', 'integrated_fitCons_score', 'GM12878_fitCons_score', 'H1-hESC_fitCons_score', 'HUVEC_fitCons_score', 'GERP++_NR', 'GERP++_RS.1', 'phyloP100way_vertebrate.1', 'phyloP30way_mammalian', 'phyloP17way_primate', 'phastCons100way_vertebrate', 'phastCons30way_mammalian', 'phastCons17way_primate', 'bStatistic', 'Interpro_domain', 'GTEx_V8_gene', 'GTEx_V8_tissue', 'ExAC_ALL', 'ExAC_AFR', 'ExAC_AMR', 'ExAC_EAS', 'ExAC_FIN', 'ExAC_NFE', 'ExAC_OTH', 'ExAC_SAS', 'MCAP', 'REVEL', 'CLNALLELEID', 'CLNDN', 'CLNDISDB', 'CLNREVSTAT', 'CLNSIG', 'cytoBand', 'avsnp147']
for dataset in datasets:
    df = pd.read_csv(os.path.join(dir_, dataset + ".hg19_multianno.csv"))
    df = df[~(df==".")]
    # print(df.shape)
    # print(len(df["GeneDetail.refGene"].value_counts()))
    # print(df["GeneDetail.refGene"])
    # print(sum(df["GeneDetail.refGene"].isna()))
    # print(df["Chr"].value_counts())
    count = df.count()
    total = df.shape[0]
    reserved = []
    for key in keys:
        if count[key] / total > 0.8:
            if key not in reserved:
                reserved.append(key)
not_reserved = list(set(keys)-set(reserved))
for dataset in datasets:
    df = pd.read_csv(os.path.join(dir_, dataset + ".hg19_multianno.csv"))
    df = df.drop(not_reserved, axis=1)
    df.to_csv(os.path.join(out_dir_, dataset + ".selected.hg19_multianno.csv"), index=False)

E:\Envs\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
E:\Envs\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (23,25,29,31,39,42,44,48,49,68,79,80,81,82,83,84,85,86) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [42]:
import pandas as pd

df = pd.DataFrame({
    "A": [1, "."],
    "B": ['.', 1]
})
print(df)
print(df[df=="."])

   A  B
0  1  .
1  .  1
     A    B
0  NaN    .
1    .  NaN


In [76]:
import pandas as pd
import os
import numpy as np

included = ['True Label', 'Chr', 'Start', 'End', 'Ref', 'Alt', 'AAChange.refGene', 'SIFT_score', 'Polyphen2_HDIV_score', 'Polyphen2_HVAR_score', 'LRT_score', 'MutationTaster_score', 'MutationAssessor_score', 'FATHMM_score', 'RadialSVM_score', 'LR_score', 'VEST3_score', 'CADD_phred']
# datasets = ['HumVarFiltered', 'ExoVarFiltered', 'VariBenchSelected', 'predictSNPSelected', 'SwissVarSelected']
dataset = 'HumVarFiltered'
input_dir = "../data/variants_selected/"
df = pd.read_csv(os.path.join(input_dir, "{}.selected.hg19_multianno.csv".format(dataset)), dtype={"True Label": int, })
print(df.info())
excluded = list(set(list(df))-set(included))
df = df.drop(excluded, axis=1)
df = df[~(df==".")]
print(df.columns)

def split(x, y):
    if not pd.isnull(x):
        trans = x.split(",")
        # first
        tran = trans[0]
        aachange = tran.split(":")
        _ = aachange[-1].split(".")[-1]
        aa_ref = _[0]
        aa_alt = _[-1]
        if y==0:
            return aa_ref
        else:
            return aa_alt
    else:
        return np.nan
   

df["A_Ref"] = df["AAChange.refGene"].apply(split, y=0)
df["A_Alt"] = df["AAChange.refGene"].apply(split, y=1)
# df["A_Alt"] = df["AAChange.refGene"].apply(split)
print(df.head())
# print(aa)
# df["A_Ref"] = aa
# df["A_Alt"] = aa[1]

for col in df.columns:
    print(col, df[col].dtypes)

E:\Envs\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AttributeError: 'Series' object has no attribute 'info'